# Projet simulation logicielle
* *Rhouch Oussama*
* *Cherki Inssaf*

<img src="figure/model based.png" alt="CS" style="width: 750px;"/>

## Importing libraries

In [1]:
import os
import librosa
import librosa.display
import numpy as np
import soundfile as sf
from scipy.io.wavfile import write
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader, Dataset
import warnings
warnings.filterwarnings('ignore')

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Performing preprocessing on the data

##### Unzip the data

In [3]:
zip_file_paths = ['data/data_part1.zip', 'data/data_part2.zip', 'data/data_part3.zip', 'data/data_part4.zip']

for zip_file_path in zip_file_paths:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(".")

##### Data folders

In [4]:
noise_folder = "data/noise/"
clean_folder = "data/clean/dev-clean"
output_folder = "data/noisy/"
denoising_folder = "data/denoising/"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)
os.makedirs(denoising_folder, exist_ok=True)

### Clean audio data

##### Load clean audio data

In [5]:
# Get the list of clean files
clean_path = []
for folder in os.listdir(clean_folder):
    folder_path = os.path.join(clean_folder, folder)
    if os.path.isdir(folder_path):
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if file.endswith(".flac"):
                    file_path = os.path.join(root, file)
                    clean_path.append(file_path)

### Noise audio data

##### Load noise audio data

In [6]:
# Get the list of noise files
noise_file = ""
if os.path.isdir(noise_folder):
        for root, dirs, files in os.walk(noise_folder):
            for file in files:
                if file.endswith(".wav"):
                    noise_file = os.path.join(root, file)

### Functions

In [7]:
def RSB(clean, noisy):
    clean = np.array(clean)
    noisy = np.array(noisy)
    return 10 * np.log10(np.sum(clean**2) / np.sum((clean - noisy)**2))

In [8]:
def alpha(RSB):
    return np.sqrt(10**(RSB/10))

In [9]:
def make_noisy(clean_file, noise_file, output_file):
    s, sr = librosa.load(clean_file, sr=None)
    u, sr = librosa.load(noise_file, sr=None)
    
    u = u[:len(s)]
    
    s_tf = np.fft.fft(s)
    u_tf = np.fft.fft(u)
    
    RSB_value = RSB(s, u)
    alpha_value = alpha(RSB_value)
    
    x_tf = s_tf + alpha_value * u_tf
    
    x = np.fft.ifft(x_tf)
    
    x = x.astype(np.float32)
    
    sf.write(output_file, x, sr)
    
    return x, s


def load_audio(file_path):
    audio, _ = librosa.load(file_path, sr=None)
    return audio

### Preprocessing

In [19]:
class SpeechDataset(Dataset):
    def __init__(self):
        self.clean_files = []
        self.noisy_files = []
        self_original_length = []
        self.i = 0
        
        for clean_file in clean_path:
            output_path = f"{output_folder}noisy_{self.i}.wav"
            x, s = make_noisy(clean_file, noise_file, output_path)
            
            self.clean_files.append(s)
            self.noisy_files.append(x)
            
            self.i += 1
            
        for i in range(len(self.clean_files)):
            s = self.clean_files[i]
            x = self.noisy_files[i]
            
            self.clean_files[i] = torch.tensor(np.abs(np.fft.fft(s)))
            self.noisy_files[i] = torch.tensor(np.abs(np.fft.fft(x)))
            
            self_original_length.append(len(s))
                
        self.max_len = max([len(s) for s in self.clean_files])
        
        for i in range(len(self.clean_files)):
            self.clean_files[i] = F.pad(self.clean_files[i], (0, self.max_len - len(self.clean_files[i])))
            self.noisy_files[i] = F.pad(self.noisy_files[i], (0, self.max_len - len(self.noisy_files[i])))
            
    def __len__(self):
        return len(self.clean_files)
    
    def __getitem__(self, idx):
        return self.noisy_files[idx], self.clean_files[idx]

In [20]:
dataset = SpeechDataset()

: 

In [12]:
input_size = len(dataset.__getitem__(0)[0])

In [13]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [14]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [15]:
#save one of the clean and noisy in files from train_loader
for i, (noisy, clean) in enumerate(train_loader):
    noisy = noisy.squeeze(0).numpy()
    clean = clean.squeeze(0).numpy()
    
    sf.write(f"{denoising_folder}noisy.wav", noisy, 16000)
    sf.write(f"{denoising_folder}clean.wav", clean, 16000)
    
    if(i==1):
        break